In [4]:
import numpy as np
import utils as util
import glob
import time
import os
import utils

class Dataset():
    def __init__(self, args):
        self.trainidx = []
        self.dataset_name = args.dataset_name
        self.path_to_annotations = os.path.join(args.annotation_path, args.dataset_name + '-Annotations/')
        self.path_to_features = os.path.join(args.I3D_path, self.dataset_name + '-I3D-JOINTFeatures.npy')
        self.labels = np.load(self.path_to_annotations + 'labels_all.npy')     # Specific to Thumos14
        self.classlist = np.load(self.path_to_annotations + 'classlist.npy')
        self.subset = np.load(self.path_to_annotations + 'subset.npy')
        self.testidx = []
        self.classwiseidx = []
        self.train_test_idx()
        self.currenttestidx = 0
        self.t_max = args.max_seqlen
        self.num_class = args.num_class
        self.classwise_feature_mapping()
        self.batch_size = args.batch_size
        self.feature_size = args.feature_size
        self.features = np.load(self.path_to_features, encoding='bytes')
        self.segments = np.load(self.path_to_annotations + 'segments.npy')
        self.labels_multihot = [util.strlist2multihot(labs,self.classlist) for labs in self.labels]


    def train_test_idx(self):
        for i, s in enumerate(self.subset):
            if s.decode('utf-8') == 'validation':   # Specific to Thumos14
                self.trainidx.append(i)
            else:
                self.testidx.append(i)

    def classwise_feature_mapping(self):
        for category in self.classlist:
            idx = []
            for i in self.trainidx:
                for label in self.labels[i]:
                    if label == category.decode('utf-8'):
                        idx.append(i); break;
            self.classwiseidx.append(idx)



In [5]:
class Args():
    def __init__(self):
        self.lr = 0.0001
        self.dataset_name = 'Thumos14reduced'
        self.num_class = 20
        self.feature_size = 2048
        self.batch_size = 24
        self.max_seqlen = 750
        self.model_name = 'weakloc'
        self.pretrained_ckpt = None
        self.max_iter = 50000
        self.num_similar = 3
        self.checkpoint_path = '/media/drive1/unsupervised_video_action_pt/checkpoint/'
        self.annotation_path = '/media/drive1/unsupervised_video_action_pt/annotations/'
        self.I3D_path = '/media/drive1/unsupervised_video_action_pt/I3D_features/'

args = Args()
self = Dataset(args)

In [13]:
annotation_path = '/media/drive1/unsupervised_video_action_pt/annotations/Thumos14reduced-Annotations/'

In [24]:
gtsegments = np.load(annotation_path + '/segments.npy')
gtlabels = np.load(annotation_path + '/labels.npy')
gtlabels = np.load(annotation_path + '/labels.npy')
videoname = np.load(annotation_path + '/videoname.npy'); videoname = np.array([v.decode('utf-8') for v in videoname])
subset = np.load(annotation_path + '/subset.npy'); subset = np.array([s.decode('utf-8') for s in subset])
classlist = np.load(annotation_path + '/classlist.npy'); classlist = np.array([c.decode('utf-8') for c in classlist])
duration = np.load(annotation_path + '/duration.npy')
ambilist = annotation_path + '/Ambiguous_test.txt'

ambilist = list(open(ambilist,'r'))
ambilist = [a.strip('\n').split(' ') for a in ambilist]

# keep training gtlabels for plotting
gtltr = []
for i, s in enumerate(subset):
    if subset[i]=='validation' and len(gtsegments[i]):
        gtltr.append(gtlabels[i])
gtlabelstr = gtltr

# Keep only the test subset annotations
gts, gtl, vn, dn = [], [], [], []
for i, s in enumerate(subset):
    if subset[i] == 'validation':
        gts.append(gtsegments[i])
        gtl.append(gtlabels[i])
        vn.append(videoname[i])
        dn.append(duration[i,0])
gtsegments = gts
gtlabels = gtl
videoname = vn
duration = dn

# keep ground truth and predictions for instances with temporal annotations
gts, gtl, vn, pred, dn = [], [], [], [], []
for i, s in enumerate(gtsegments):
    if len(s):
        gts.append(gtsegments[i])
        gtl.append(gtlabels[i])
        vn.append(videoname[i])
        dn.append(duration[i])
gtsegments = gts

In [46]:
duration[12]

171.439

In [28]:
gtsegments[12]

[[4.3, 10.9],
 [16.3, 21.1],
 [28.5, 36.8],
 [56.2, 73.0],
 [85.7, 101.0],
 [121.4, 141.6]]

In [43]:
feats[12][0:240]

array([[0.16569176, 0.04577682, 0.12028474, ..., 0.011098  , 0.02648151,
        0.42048913],
       [0.18735085, 0.08297902, 0.1321953 , ..., 0.10974882, 0.01641319,
        0.        ],
       [0.27668834, 0.06708378, 0.13456516, ..., 0.31083372, 0.        ,
        0.13812083],
       ...,
       [0.17345142, 0.10697565, 0.13193522, ..., 0.27103055, 0.9397692 ,
        0.02320028],
       [0.17254564, 0.13406193, 0.11197495, ..., 0.17237894, 0.23642866,
        0.03399808],
       [0.15892404, 0.24255729, 0.18422672, ..., 0.00848267, 0.17708796,
        0.18079375]], dtype=float32)